In [ ]:
# COOKIE
> Javascript Cookies

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: 

## What are Cookies

Cookies are information that is stored locally in the browser. Since it is stored locally, the data will still be there even if you close the browser or reload the session. 

However, there are some cons of cookies. Namely, there isn't a lot of storage space allocated for cookies. Cookies are also a privacy concern because they store potentially private information. 

## Main uses of Cookies in the case of a game

In this game, we used cookies to store the scores of each play through. In order to do this, we need to know how to create cookies, as well as read cookies. 

In [ ]:
document.cookie = "cookiename=value1";

### Creating Cookies

The "cookiename" is the name of the cookie, and the "value1" is the value of the cookie. However, by default, the session will delete the cookie when you close the browser. In order to stop this, a expiration date is needed. 

In [ ]:
document.cookie = "cookiename=value1; expires=Thu, 18 Dec 2038 12:00:00 UTC";

### Reading Cookies


A variable can be set to the name and the value of the cookie. In this example, the name, value, and the expiration date are all variables. 

When a variable is set to document.cookie, it will return all cookies in the way of "cookie1=value1; cookie2=value2;" and so on.

In [ ]:
let x = document.cookie = cookieName + '=' + cookieValue + '; expires=' + daysToExpire + ';SameSite=None';

var cookieString = document.cookie; //returns all cookies


### Deleting cookies

To delete a cookie, just set the expiration date to a time before the current time. 

function deleteCookie(cookieName) {
      document.cookie = cookieName + '=; expires=Thu, 01 Jan 1970 00:00:00 UTC; path=/;';
  }

### Creating a table

To create a table containing all the cookie values, you would want to create a for loop that loops through all the values stored. After that, you create a column for the name, and another column for the value. 


In [ ]:
var tableBody = document.querySelector('#tablearr'); //pushes onto the table


function tablegen() {
    for (var i = 0; i < cookies.length; i++) {
      var cookie = cookies[i].trim().split('='); //trim and split it up before and after the equal sign
      var name = cookie[0]; //first half
      var value = cookie[1]; //second half
      
      var row = document.createElement('tr');
      var nameCell = document.createElement('td');
      nameCell.textContent = name;
      row.appendChild(nameCell); //adds the item
  
      var valueCell = document.createElement('td');
      valueCell.textContent = value; //value of the cookie
      row.appendChild(valueCell);
      tableBody.appendChild(row);
      
      var deleteButtonCell = document.createElement('td');
      var deleteButton = document.createElement('button'); //the butotn
      deleteButton.textContent = 'rmove'; //text in button
      deleteButton.addEventListener('click', function()
      {
        deleteCookie(name); //functino so that it doesn't run automatically
      });
      deleteButtonCell.appendChild(deleteButton);
      row.appendChild(deleteButtonCell); //these 2 add the button
    }
  }

<table id="tablearr">
  <!-- Table headers or initial content -->
  <tr>
    <th>Session</th>
    <th>Score</th> 
    <!-- in this example, there are only 2 columns, but if you wanted to have a third for the delete, just create another <th> tag.  -->
  </tr>
</table>


### High Score

In the game, we had a high score function. We just created a cookie highscore, that everytime that the game reset, it would check if the current score was greater than the highscore, in which we replaced the highscore with that score. 

In order to return the high score, we had to have a function that would look for that highscore.

In [ ]:
function getHighScore() {
    var cookieString = document.cookie; //returns all cookies
    var cookieArray = cookieString.split(";"); //splits by ;, so it goes by each cookie.
  
    // Iterate over each cookie
    for (var i = 0; i < cookieArray.length; i++) {
      var cookie = cookieArray[i].trim();
  
      // Check if the cookie starts with the provided name
      if (cookie.startsWith("highscore=")) {
        // Extract and return the cookie value
        var score = parseInt(cookie.substring("highscore=".length));    
        return score;
        }
    }
  
    // Return null if the cookie is not found
    console.log("notfound");
    return null;
  }